In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
batch_size = 32  # The default batch size of keras.
num_classes = 10  # Number of class for the dataset
epochs = 100

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170498071/170498071 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices. This is called one hot encoding.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [4]:
def efficonvres(projection_dim):

    def apply(inputs):
        x_input = inputs

        x = layers.Conv2D(filters=projection_dim,kernel_size=3,padding="same",groups=projection_dim)(x_input)

        x1 = layers.DepthwiseConv2D(3, padding="same", use_bias=False)(x)
        x1 = layers.BatchNormalization(3)(x1)
        x1 = layers.Activation("swish")(x1)

        x1 = layers.DepthwiseConv2D(3, padding="same", use_bias=False)(x1)
        x1 = layers.LayerNormalization(epsilon=1e-6)(x1)
        x1 = layers.Activation("gelu")(x1)

        x = layers.LayerNormalization(epsilon=1e-6)(x)
        x = layers.Dense(4 * projection_dim)(x)
        x = layers.Activation("gelu")(x)
        x = layers.Dense(projection_dim)(x)

        x = tf.keras.layers.Add()([x, x1])
        return x
    return apply

dims = [32, 64, 128, 256]
inputs = tf.keras.Input(shape=(32,32,3))
x = layers.Conv2D(filters=128,kernel_size=3,padding="same", activation='gelu')(inputs)
for i in range(4):
    x = layers.Conv2D(filters=dims[i],kernel_size=3,padding="same", activation='gelu')(x)
    x = efficonvres(dims[i])(x)
x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
output = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=output)

In [5]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [6]:
model.fit(x_train, y_train, epochs=50, batch_size=64, 
                                  validation_data=(x_test, y_test))

Epoch 1/50
782/782 [==============================] - 196s 223ms/step - loss: 1.4196 - accuracy: 0.4800 - val_loss: 1.2363 - val_accuracy: 0.5526
Epoch 2/50
782/782 [==============================] - 171s 219ms/step - loss: 1.0622 - accuracy: 0.6196 - val_loss: 1.0066 - val_accuracy: 0.6367
Epoch 3/50
782/782 [==============================] - 171s 219ms/step - loss: 0.9022 - accuracy: 0.6785 - val_loss: 1.0931 - val_accuracy: 0.6076
Epoch 4/50
782/782 [==============================] - 171s 219ms/step - loss: 0.7952 - accuracy: 0.7197 - val_loss: 0.8396 - val_accuracy: 0.7040
Epoch 5/50
782/782 [==============================] - 171s 218ms/step - loss: 0.7102 - accuracy: 0.7496 - val_loss: 0.7629 - val_accuracy: 0.7265
Epoch 6/50
782/782 [==============================] - 171s 219ms/step - loss: 0.6417 - accuracy: 0.7733 - val_loss: 0.7435 - val_accuracy: 0.7421
Epoch 7/50
782/782 [==============================] - 171s 218ms/step - loss: 0.5792 - accuracy: 0.7968 - val_loss: 0.6937 -

In [7]:
model.save("efficonvres_10.h5")